<a href="https://colab.research.google.com/github/nalewkoz/aidit/blob/main/trim_video_trigger_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is this anyway?
The code belows finds the first occurrence of a trigger frame and trims off the intro part of the video up until that frame.

You can edit the link to the trigger frame to change what prompts the trim. The provided function `create_average_start_frame` can be used to distill the trigger frame from exemplary videos.

## Step-by-step guide

1. Click the play button below.
2. Click "Choose files".
3. Choose the files to trim.
4. Wait a few minutes. 

After all the video files are uploaded they are processed one by one. The download should start automatically.

In [ ]:
#@title
'''
TO DO:
-- progress bar
-- easy update trigger frame ?
-- trigger souns (speech, or specific words)
'''
import cv2
import numpy as np
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
import time
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip

link_trigger_frame = "https://github.com/nalewkoz/aidit/blob/main/trigger_frame.png?raw=true"

path_start       = "/content/drive/MyDrive/"
path_start_video = ""

debug = False

def create_average_start_frame(videos_names, out_name='average_frame.png', Nframes = 100):
    drive.mount('/content/drive')

    av_frame = None
    Nav      = 0
    for v_name in videos_names:
        video_path = path_start_video+v_name
        print("Reading "+ video_path + " ...")

        cap = cv2.VideoCapture(video_path)

        if cap.isOpened()== False:
            print("Error opening the file: "+ video_path)
    
        Nfr = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                Nav += 1
                Nfr += 1
                if av_frame is None:
                    av_frame = frame.astype(np.float32)
                else:
                    av_frame += frame 
            else:
                break
            if Nfr >= Nframes:
                break
        cap.release()

    # save the file    
    av_frame = av_frame/Nav
    im_path = path_start+out_name
    print("Saving " + im_path + " ...")
    cv2.imwrite(im_path, av_frame)

def find_start_time(in_name, out_name = None, av_image_name = None, t0 = 100, dt = 10000, dt2 = 100, dl2_th = 10000, skip_pixels=5):
    
    in_path       = path_start_video + in_name
    if av_image_name is None:
        !wget -nc $link_trigger_frame -O trigger.png
        av_image_path = "trigger.png"
    else:
        av_image_path = path_start + av_image_name

    av_image = cv2.imread(av_image_path)

    av_image_rescaled = av_image[::skip_pixels,::skip_pixels].astype(np.float32)

    Npixels = av_image_rescaled.shape[0]*av_image_rescaled.shape[1]

    t_hist  = []
    l2_hist = []

    print("Reading "+ in_path + " ...")
    print("Image resolution: ")
    print(av_image.shape)

    cap = cv2.VideoCapture(in_path)
    t = t0

    if cap.isOpened()== False:
        print("Error opening the file: "+ in_path)
    
    l2_old = None

    while cap.isOpened():
        cap.set(cv2.CAP_PROP_POS_MSEC, t)
        ret, frame = cap.read()
        if ret == True:
            l2  = np.sum( (frame[::skip_pixels,::skip_pixels].astype(np.float32) - av_image_rescaled)**2)/Npixels
            l2_hist.append(l2) 
            t_hist.append(t)

            if l2_old is None:
                if debug:
                    cv2.imwrite("trial.png", frame[::skip_pixels,::skip_pixels])
                    files.download("trial.png")
                l2_old = l2
            else:
                if l2_old - l2 > dl2_th:
                    break
            t += dt
        else:
            break
    
    t = t - dt + dt2

    while cap.isOpened():
        cap.set(cv2.CAP_PROP_POS_MSEC, t)
        ret, frame = cap.read()
        if ret == True:
            l2  = np.sum( (frame[::skip_pixels,::skip_pixels].astype(np.float32) - av_image_rescaled)**2)/Npixels
            l2_hist.append(l2) 
            t_hist.append(t)

            if l2_old is None:
                l2_old = l2
            else:
                if l2_old - l2 > dl2_th:
                    break
            t += dt2
        else:
            break
     
    cap.release()

    if debug:
        fig = plt.figure()
        plt.plot(t_hist, l2_hist, '.')
        plt.xlabel('frame #')
        plt.ylabel('L2 distance from the trigger frame')
    print(f"I will trim initial {t/1000:.2f} seconds.")

    return t

def cut_video(in_name, t_start, out_name = None ):
    if out_name is None:
        out_name = in_name[:-4] + "_cut_intro" + in_name[-4:]
    
    in_path       = path_start_video + in_name
    out_path      = path_start_video + out_name
    
    # loading video 
    clip = VideoFileClip(in_path)
    
    # getting duration of the video
    duration = clip.duration

    ffmpeg_extract_subclip(in_path, t_start/1000, duration, targetname=out_path)

    return out_path

uploaded = files.upload()

for in_name in uploaded.keys():
    print("File: " + in_name)
    print("Finding where to clip...")
    start=time.time()
    t_start_ms = find_start_time(in_name)
    stop=time.time()
    print(f"Execution time in seconds: {stop-start}")
    
    print("Preparing clipped video...")
    start=time.time()
    fname = cut_video(in_name, t_start_ms)
    stop=time.time()
    print(f"Execution time in seconds: {stop-start}")
    files.download(fname)